In [1]:
import subprocess
repo_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')

In [2]:
import sys 
sys.path.append(repo_dir) # go to parent dir from customFunctions import *
import statbot_helpers as sbh
from pyaxis import pyaxis
import pandas as pd
import numpy as np
import os
import requests
import json

In [3]:
# bfs_nr of dataset
BFS_NR = 'px-x-0102010000_101'

STICHTAG =  '-12-31' # -%m-%d

# get asset_nr
asset_nr = sbh.get_bfs_asset_nr(BFS_NR)
BFS_URL = "https://www.bfs.admin.ch/bfsstatic/dam/assets/" + str(asset_nr) + "/master"

file_name="bfs_1_01_001_CH.px"

# download cube
r = requests.get(BFS_URL)
open(os.path.join(repo_dir,"temp/"+file_name), 'wb').write(r.content)

108324271

In [4]:
# load data from bfs
px = pyaxis.parse(os.path.join(repo_dir,"temp/"+file_name), encoding='ISO-8859-2')

In [5]:
# clean df
df = px['DATA'].loc[px['DATA']['Bevölkerungstyp'] == 'Ständige Wohnbevölkerung']
df = df.drop(columns=['Bevölkerungstyp'])
df = df.rename(columns={"Kanton (-) / Bezirk (>>) / Gemeinde (......)": "name"})


# add column with spatialunit_ontology
df.loc[df['name'].str.startswith("......"), "spatialunit_ontology"] = "A.ADM3"
#df.loc[df['name'].str.startswith(">>"), "spatialunit_ontology"] = "A.ADM2"
#df.loc[df['name'].str.startswith("-"), "spatialunit_ontology"] = "A.ADM1"
#df.loc[df['name'].str.startswith("Schweiz"), "spatialunit_ontology"] = "CH"

df = df.dropna(subset=['spatialunit_ontology'])

df['spatialunit_ontology'] = df['spatialunit_ontology'].astype('category') # reduce memory footprint


# extract bfs_nr and name for gemeinde
df.loc[df['spatialunit_ontology'] == 'A.ADM3', 'bfs_nr'] = df['name'].str.slice(6,11)
df.loc[df['spatialunit_ontology'] == 'A.ADM3', 'name'] = df['name'].str.slice(11)

df['bfs_nr'] = df['bfs_nr'].astype('int16')

# extract name for bezirk
#df.loc[df['spatialunit_ontology'] == 'A.ADM2', 'name'] = df['name'].str.slice(3)

# extract name for kanton
#df.loc[df['spatialunit_ontology'] == 'A.ADM1', 'name'] = df['name'].str.slice(2)

# convert origin
df.loc[df['Staatsangehörigkeit (Kategorie)'] == 'Staatsangehörigkeit (Kategorie) - Total', 'Staatsangehörigkeit (Kategorie)'] = '-1'
df.loc[df['Staatsangehörigkeit (Kategorie)'] == 'Schweiz', 'Staatsangehörigkeit (Kategorie)'] = '1'
df.loc[df['Staatsangehörigkeit (Kategorie)'] == 'Ausland', 'Staatsangehörigkeit (Kategorie)'] = '2'

# convert sex
df.loc[df['Geschlecht'] == 'Geschlecht - Total', 'Geschlecht'] = '-1'
df.loc[df['Geschlecht'] == 'Mann', 'Geschlecht'] = '1'
df.loc[df['Geschlecht'] == 'Frau', 'Geschlecht'] = '2'

df['Geschlecht'] = df['Geschlecht'].astype('category') # reduce memory footprint


# convert age
df['Alter'] = df['Alter'].str.split(" ", n=1, expand=True)[0]
df.loc[df['Alter'] == 'Alter', 'Alter'] = '-1'

df['DATA'] = df['DATA'].astype('int32')
df['Alter'] = df['Alter'].astype('int16')

maximum=max(df['Jahr'])
df['Jahr'] += STICHTAG

df['period_value'] = np.NAN

df.columns = ['time_value', 'spatialunit_name', 'origin', 'sex', 'age', 
              'value', 'spatialunit_ontology', 'spatialunit_current_id', 'period_value']

df = df[['spatialunit_ontology', 'spatialunit_name', 
         'time_value', 'period_value', 'value', 'origin', 'sex', 'age', 'spatialunit_current_id']]


In [6]:
spatial_reference=str(maximum)+"-10-18"
#spatial_reference

df_clean = sbh.convert_current_to_hist_id(df, spatial_reference, os.path.join(repo_dir,"data/spatialunits.csv"))

df_try = df_clean.copy()

#print(df_try.shape)

#print(df_try['spatialunit_hist_id'].isnull().sum())

list_ontologies = ['A.ADM2', 'A.ADM1', 'CH']

list_dimensions = ['origin', 'sex', 'age']

#df_adm2 = sbh.add_granularity_levels_up(df_try,list_ontologies, repo_dir, list_dimensions)

df_new = sbh.add_granularity_levels_up(df_try, list_ontologies, repo_dir, list_dimensions)

/home/b105p02@ji.ktzh.ch/gitrepos/statbot/statbot_helpers/translate_to_spatial_unit_name.py:11: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  df_temp = pd.merge(df, spatial_unit_table, how='left', on=['spatialunit_ontology', 'spatialunit_hist_id'])


In [7]:
#df_new[df_new['spatialunit_name'].isnull()]['spatialunit_hist_id'].unique()

array([10061, 10070, 10093, 10108, 10109, 10110, 10187, 10252, 10320])

In [12]:
#df_new

,spatialunit_ontology,spatialunit_name,time_value,period_value,value,origin,sex,age,spatialunit_hist_id
0,A.ADM3,Aeugst am Albis,2010-12-31,NaN,1824,-1,-1,-1,13256
1,A.ADM3,Aeugst am Albis,2010-12-31,NaN,14,-1,-1,0,13256
2,A.ADM3,Aeugst am Albis,2010-12-31,NaN,18,-1,-1,1,13256
3,A.ADM3,Aeugst am Albis,2010-12-31,NaN,13,-1,-1,2,13256
4,A.ADM3,Aeugst am Albis,2010-12-31,NaN,20,-1,-1,3,13256
...,...,...,...,...,...,...,...,...,...
23649511,CH,Schweiz,2020-12-31,NaN,165,2,2,96,0
23649512,CH,Schweiz,2020-12-31,NaN,127,2,2,97,0
23649513,CH,Schweiz,2020-12-31,NaN,77,2,2,98,0
23649514,CH,Schweiz,2020-12-31,NaN,51,2,2,99,0


In [13]:
df_new.to_csv(os.path.join(repo_dir,"data/11_11001.csv"), index=False, na_rep='NA')

sys.exit(0)


SystemExit: 0

/home/b105p02@ji.ktzh.ch/gitrepos/statbot/statbot_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
